In [ ]:
import pandas as pd
from IPython.display import Image

In [ ]:
df_test = pd.read_csv('data/input/raw/test.csv')
df_train = pd.read_csv('data/input/raw/train.csv')

df_test.example_path = df_test.example_path.str.replace('train_test_data','data/input/raw')
df_train.example_path = df_train.example_path.str.replace('train_test_data','data/input/raw')

df_train.head()

In [ ]:
filename = df_train.iloc[0].example_path

Image(filename=filename) 